In [18]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
df = pd.read_csv('dataset\Grocery Products Purchase.csv')

<>:1: DeprecationWarning: invalid escape sequence '\G'
<>:1: DeprecationWarning: invalid escape sequence '\G'
C:\Users\Rudini\AppData\Local\Temp\ipykernel_2756\4051947293.py:1: DeprecationWarning: invalid escape sequence '\G'
  df = pd.read_csv('dataset\Grocery Products Purchase.csv')


In [3]:
df.describe()

,Product 1,Product 2,Product 3,Product 4,Product 5,Product 6,Product 7,Product 8,Product 9,Product 10,...,Product 23,Product 24,Product 25,Product 26,Product 27,Product 28,Product 29,Product 30,Product 31,Product 32
count,9835,7676,6033,4734,3729,2874,2229,1684,1246,896,...,14,8,7,7,6,5,4,1,1,1
unique,158,151,155,153,150,137,138,140,128,120,...,13,7,6,7,5,4,3,1,1,1
top,sausage,whole milk,whole milk,whole milk,rolls/buns,soda,soda,shopping bags,soda,shopping bags,...,waffles,bottled beer,chocolate,shopping bags,hygiene articles,napkins,house keeping products,skin care,hygiene articles,candles
freq,825,654,506,315,176,150,120,76,61,49,...,2,2,2,1,2,2,2,1,1,1


In [4]:
def retrieve_data(df):
    
    txs = []
    
    for transaction in df.values:
        tx = []

        #filtering non null values
        for item in transaction:
            if type(item) != float:
                tx.append(item.lower())

        # print(tx) 
        txs.append(tx)

    return txs

In [5]:
transactions = retrieve_data(df) #list of lists for pre-processed data for apriori implementation

save to csv file (supposedly to be added to database)

In [6]:
# df = pd.DataFrame(retrieve_data(df), columns=df.columns)
# df.to_csv('processed_data\\new_transaction.csv', index=False) #option to save to csv file the pre-processed data

APRIORI INTEGRATION

In [7]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
# df

In [8]:
min_support = 0.01 #1% support threshold, adjust if needed to retrieve meaningful data

In [9]:
frequent_itemsets = apriori(df, min_support, use_colnames=True)
frequent_itemsets['itemsets'] = frequent_itemsets['itemsets'].apply(lambda x: [item.lower() for item in list(x)])

In [10]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules = rules[(rules['lift'] > 1.2) & (rules['confidence'] >= 0.5)]
rules
# for index, row in rules.iterrows():
#     print(row['antecedents'], row['consequents'])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
414,"(other vegetables, butter)",(whole milk),0.020031,0.255516,0.011490,0.573604,2.244885,0.006371,1.745992,0.565878
418,"(root vegetables, citrus fruit)",(other vegetables),0.017692,0.193493,0.010371,0.586207,3.029608,0.006948,1.949059,0.681990
438,"(curd, yogurt)",(whole milk),0.017285,0.255516,0.010066,0.582353,2.279125,0.005649,1.782567,0.571107
444,"(domestic eggs, other vegetables)",(whole milk),0.022267,0.255516,0.012303,0.552511,2.162336,0.006613,1.663694,0.549779
462,"(pip fruit, other vegetables)",(whole milk),0.026131,0.255516,0.013523,0.517510,2.025351,0.006846,1.543003,0.519843
472,"(rolls/buns, root vegetables)",(other vegetables),0.024301,0.193493,0.012201,0.502092,2.594890,0.007499,1.619792,0.629935
490,"(root vegetables, tropical fruit)",(other vegetables),0.021047,0.193493,0.012303,0.584541,3.020999,0.008231,1.941244,0.683367
502,"(root vegetables, yogurt)",(other vegetables),0.025826,0.193493,0.012913,0.500000,2.584078,0.007916,1.613015,0.629266
534,"(whipped/sour cream, other vegetables)",(whole milk),0.028876,0.255516,0.014642,0.507042,1.984385,0.007263,1.510239,0.510816
546,"(yogurt, other vegetables)",(whole milk),0.043416,0.255516,0.022267,0.512881,2.007235,0.011174,1.528340,0.524577


In [11]:
def recommend_item(itemset, rules):
    
    # recos = set()
    # print(itemset)
    print(f"cart item/s: {itemset}")

    recos = set()

    def check_rules(item):

        for index, row in rules.iterrows():

            antecedents = list(row['antecedents'])
            consequents = list(row['consequents'])

            if item in antecedents or item in consequents:

                for items in antecedents:
                    if items != item and items not in itemset and items not in recos:
                        recos.add(items)

                for items in consequents:
                    if items != item and items not in itemset and items not in recos:
                        recos.add(items)
    
    for item in itemset:
        check_rules(item)

    return recos
    #return list(recos)

In [17]:
user_transaction = ['domestic eggs', 'whole milk']
recommend_item(user_transaction, rules)

cart item/s: ['domestic eggs', 'whole milk']


{'butter',
 'curd',
 'other vegetables',
 'pip fruit',
 'rolls/buns',
 'root vegetables',
 'tropical fruit',
 'whipped/sour cream',
 'yogurt'}